<a href="https://colab.research.google.com/github/VictorHugoMartins/israel_x_palestine_data_analysis/blob/main/preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Limpeza de Comentários

Redução do vocabulário dos comentários coletados. Vídeos e Canais não passam por esse processo por não serem o foco da pesquisa e servirem como bases auxiliares para o percurso principal.

In [39]:
from config import result_index

execution = 'local'
# execution = 'cloud'

base_path = 'data' if execution == 'local' else 'cloud'

In [40]:
import pandas as pd
import os
from glob import glob

# Diretórios
videos_dir = f"processed_data_{result_index}"

# Função para ler e concatenar todos os CSVs de um diretório
def load_csvs_from_directory(directory):
    all_files = glob(os.path.join(directory, "*.csv"))  # Lista todos os arquivos CSV
    df_list = [pd.read_csv(file) for file in all_files]  # Lê cada CSV e adiciona à lista
    return pd.concat(df_list, ignore_index=True) if df_list else pd.DataFrame()  # Concatena todos os DataFrames

# Carregar os DataFrames completos
df_videos = load_csvs_from_directory(videos_dir)

# Exibir quantidades filtradas
print(f"qtd videos: {len(df_videos)}")

qtd videos: 935


In [41]:
unified_data = pd.concat([df_videos], ignore_index=True)
unified_data.sample(5)

,Unnamed: 0.1,Unnamed: 0,video_id,title,description,channel_id,published_at,category_id,tags,view_count,...,concurrent_viewers,active_live_chat_id,recording_date,topicCategories,processing_status,parts_total,parts_processed,time_left_ms,processing_failure_reason,text
602,NaN,NaN,rJ4VBKnYPqU,A Mulher de valor vai multiplicar o que recebe...,#rafaelaires #antiotario #redpillbrasil\n\n🔥 P...,UCAYoI16-UkXemcnhC-kTvDQ,2025-05-21 12:01:40,22,[],21472,...,0,NaN,NaN,[],NaN,0,0,0,NaN,NaN
587,NaN,NaN,aUz37VXcMv4,A verdadeira amizade resiste ao tempo... Quand...,#rafaelaires #antiotario #redpillbrasil\n\n🔥 P...,UCAYoI16-UkXemcnhC-kTvDQ,2025-05-22 15:00:04,22,[],11385,...,0,NaN,NaN,[],NaN,0,0,0,NaN,NaN
288,288.0,319.0,0uNW4Fcwf-4,Maturidade não é sobre aceitar qualquer coisa ...,#rafaelaires #antiotario #redpillbrasil\n\n🔥 P...,UCAYoI16-UkXemcnhC-kTvDQ,2025-05-10 15:00:03,22,[],280225,...,0,NaN,NaN,['https://en.wikipedia.org/wiki/Lifestyle_(soc...,NaN,0,0,0,NaN,maturidade não é sobre aceitar qualquer coisa ...
842,NaN,NaN,VkLD95HYg4M,O LADO SOMBRIO DO SHOW DA LADY GAGA! - part. E...,O LADO SOMBRIO DO SHOW DA LADY GAGA! - part. E...,UCeL1a4rpEA8UG9IQIewPccg,2025-05-06 03:03:11,22,"['podcast', 'entrevista', 'lady gaga', 'show',...",41144,...,0,NaN,NaN,['https://en.wikipedia.org/wiki/Entertainment'],NaN,0,0,0,NaN,NaN
651,NaN,NaN,Gqas4ZUno0k,AS REDES SOCIAIS SÃO REMÉDIOS TARJA PRETA PARA...,Neste vídeo pretendi deixar de lado as várias ...,UC3nQ4xUl6rodOWuQbBULyow,2025-05-18 00:39:57,24,"['don sandro', 'entretenimento', 'baladas']",9342,...,0,NaN,NaN,[],NaN,0,0,0,NaN,NaN


In [42]:
# Função para remover duplicatas e manter os registros mais recentes.
def remove_duplicates(df, id_column, date_column):
    # Ordena por data de forma decrescente e remove duplicatas mantendo o mais recente
    df_sorted = df.sort_values(by=date_column, ascending=False)
    df_unique = df_sorted.drop_duplicates(subset=id_column, keep='first')
    return df_unique

In [43]:
# Gerar as estatísticas
unified_data = remove_duplicates(unified_data, 'video_id', 'published_at')

In [44]:
unified_data = unified_data.dropna(subset=["video_id", "description", "published_at", "channel_id", "title"])

In [45]:
unified_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 447 entries, 0 to 446
Data columns (total 38 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Unnamed: 0.1               203 non-null    float64
 1   Unnamed: 0                 203 non-null    float64
 2   video_id                   447 non-null    object 
 3   title                      447 non-null    object 
 4   description                447 non-null    object 
 5   channel_id                 447 non-null    object 
 6   published_at               447 non-null    object 
 7   category_id                447 non-null    int64  
 8   tags                       447 non-null    object 
 9   view_count                 447 non-null    int64  
 10  like_count                 447 non-null    int64  
 11  comment_count              447 non-null    int64  
 12  duration                   447 non-null    object 
 13  definition                 447 non-null    object 
 14 

In [75]:
unified_data["text"] = unified_data["title"].fillna('') + " " + unified_data["description"].fillna('')

## Filtragem por idioma

Apenas comentários em inglês serão utilizados na pesquisa. Não haverá, por agora, uma abordagem multilanguage.

In [76]:
# # 1. Instalar FastText
%pip install fasttext-wheel

# # 2. Importar bibliotecas necessárias
import fasttext

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\vmart\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [77]:
# # 3. Baixar o modelo de identificação de idiomas do FastText
# # Use the raw file URL to download the binary model directly
import requests

def download_model(url, output_path):
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()
        with open(output_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print(f"Modelo baixado com sucesso: {output_path}")
    except Exception as e:
        print(f"Erro ao baixar o modelo: {e}")

# # URL do modelo e caminho de saída
url = "https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin"
output_path = "data/models/lid.176.bin"

# # Faz o download do modelo
download_model(url, output_path)

Modelo baixado com sucesso: data/models/lid.176.bin


In [78]:
# # Carregae o modelo de linguagem
def load_model():
    model_path = 'data/models/lid.176.bin'
    model = fasttext.load_model(model_path)
    return model

model = load_model()

# # Função para identificar o idioma de um comentário
def detect_language(comment):
    # Remover quebras de linha
    if isinstance(comment, str):
      comment = comment.replace('\n', ' ').strip()  # Remove quebras de linha e espaços extras
      prediction = model.predict(comment)
      return prediction[0][0]  # Retornar o código do idioma
    else:
      return ''


def filter_by_english(df, verbose=False):
  #   # Aplicar a função de detecção de idioma a cada comentário
  df['language'] = df['text'].apply(detect_language)

  print(df['language'])

  # Filtrar os comentários em inglês
  english_comments = df[df['language'] == '__label__en']

  return english_comments

In [79]:
unified_data = filter_by_english(unified_data, verbose=True)

unified_data[['video_id', 'text']].sample(5)

ValueError: Unable to avoid copy while creating an array as requested.
If using `np.array(obj, copy=False)` replace it with `np.asarray(obj)` to allow a copy when needed (no behavior change in NumPy 1.x).
For more details, see https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword.

## Limpeza Textual


In [80]:
from utils.clean_data import clean_data

unified_data = clean_data(unified_data, 'video_id', 'text')

Removendo duplicatas...
Limpando texto...
Removendo stopwords...
Lematizando...
Removendo linhas vazias...
Filtrando por tamanho do texto...


## Visualizações

In [81]:
unified_data.sample(5)

,Unnamed: 0.1,Unnamed: 0,video_id,title,description,channel_id,published_at,category_id,tags,view_count,...,concurrent_viewers,active_live_chat_id,recording_date,topicCategories,processing_status,parts_total,parts_processed,time_left_ms,processing_failure_reason,text
216,216.0,242.0,mlZKX6aUrYs,"Hoje em dia, o emocionado pede pra ser corno c...",#rafaelaires #antiotario #redpillbrasil\n\n🔥 P...,UCAYoI16-UkXemcnhC-kTvDQ,2025-05-15 21:00:46,22,[],24726,...,0,NaN,NaN,['https://en.wikipedia.org/wiki/Lifestyle_(soc...,NaN,0,0,0,NaN,hoje em dia emocionado pede pra ser corno com ...
722,NaN,NaN,wQsocc0h0y8,MULHER FALA SOBRE AS QUE PAGA COISAS PARA OS H...,"Olá! seja bem vindo a mais um video do canal, ...",UCRmNflJuD1TxLbRlDV08_7g,2025-05-14 00:59:48,24,"['red pill', 'divorcio', 'pensão socio afetiva...",2000,...,0,NaN,NaN,"['https://en.wikipedia.org/wiki/Health', 'http...",NaN,0,0,0,NaN,mulher fala sobre que paga coisas para o homen...
400,400.0,440.0,PnlROGiwzXQ,PAZ! O que o Homem mais quer no casamento. | Q...,#rafaelaires #antiotario #redpillbrasil\n\n🔥 P...,UCAYoI16-UkXemcnhC-kTvDQ,2025-05-03 21:00:20,22,[],47565,...,0,NaN,NaN,['https://en.wikipedia.org/wiki/Lifestyle_(soc...,NaN,0,0,0,NaN,paz que homem mais quer casamento qual sua opi...
548,NaN,NaN,0KDV-o-hKMA,Levou alimentos pros filhos e a ex reclamou: q...,#rafaelaires #antiotario #redpillbrasil\n\n🔥 P...,UCAYoI16-UkXemcnhC-kTvDQ,2025-05-24 21:00:00,22,[],112,...,0,NaN,NaN,['https://en.wikipedia.org/wiki/Society'],NaN,0,0,0,NaN,levou alimentos pro filhos e ex reclamou queri...
920,NaN,NaN,on01uAZM4Jw,MULHER FALA QUE É MENTIRA QUE GAROTA DE 20 ANO...,seja bem vindo a mais um vídeo do canal Não se...,UCUBeVY6Kn7ulBmUGynJqISw,2025-05-01 23:56:55,24,"['redpill', 'msol', 'miqueinha', 'relacionamen...",840,...,0,NaN,NaN,['https://en.wikipedia.org/wiki/Humour'],NaN,0,0,0,NaN,mulher fala que é mentira que garota de ano go...


In [82]:
# Contar a quantidade de vídeos por autor
channel_id_counts = unified_data['channel_id'].value_counts()

# Selecionar os 10 autores com maior contagem
top_10_channel_ids = channel_id_counts.nlargest(10).index

# Filtrar o DataFrame mantendo apenas os vídeos desses autores
unified_data = unified_data[unified_data['channel_id'].isin(top_10_channel_ids)]

unified_data['channel_id'].value_counts()

channel_id
UCAYoI16-UkXemcnhC-kTvDQ    176
UCRmNflJuD1TxLbRlDV08_7g     69
UCO9FRrBUwGdYopkMbGGKbpg     46
UCeL1a4rpEA8UG9IQIewPccg     41
UCUBeVY6Kn7ulBmUGynJqISw     39
UCNiU1wZxK6YN-KuJP7QMpBQ     32
UCX0VSzJ2z5l0C9wnwh5SoRw     23
UC3nQ4xUl6rodOWuQbBULyow     18
UCExFA9MsrRmWnXUlhiwu4qA      3
Name: count, dtype: int64

# Exportação de Dados

In [83]:
unified_data.to_csv(f'processed_data_{result_index}/cleaned_videos.csv', encoding='utf-8')

## Equivalentes

In [84]:
processed_data_dir = f"processed_data_{result_index}"
os.makedirs(processed_data_dir, exist_ok=True)  # Cria o diretório se não existir

results_data_dir = f'results_{result_index}'
os.makedirs(results_data_dir, exist_ok=True)  # Cria o diretório se não existir